In [42]:
import requests
import keyboard
import time
import os
from keras.preprocessing import image
import win32com.client as wincl
import speech_recognition as sr

# initialize the Keras REST API endpoint URL along with the input
# image path
KERAS_REST_API_URL = "http://localhost:5000/predict"
IMAGE_PATH = "gesture.jpg"


def get_assistant(sentence):
    speak = wincl.Dispatch("SAPI.SpVoice")
    r = sr.Recognizer()
    speak.Speak(sentence)
    with sr.Microphone() as source:
        print("listening..")
        audio = r.listen(source)
        reply = r.recognize_google(audio)
    print("Speech to Text: ", reply )

    return reply

def detect_sign_language():
    pred=""
    sentence=""
    sentence_all=[]
    put_space = False
    read_end = time.time()
    while True:
        read_start = time.time()
        if read_start-read_end>10:
            put_space = True
        if keyboard.is_pressed('Esc'):
            # 'ESC' pressed
            print("Escape hit, closing...")
            break
        image_modified_time= os.path.getmtime(IMAGE_PATH)
        cur_time = time.time()
        time_diff = cur_time-image_modified_time
        try:
            image = open(IMAGE_PATH, "rb").read()
            img_backup = image

        except:
            img = img_backup
        if time_diff<1:
            frames = frames+1
            payload = {"image": image}
            try:
                if frames>10:
                    # submit the request
                    r = requests.post(KERAS_REST_API_URL, files=payload).json()
                    # ensure the request was sucessful
                    if r["success"]:
                        # loop over the predictions and display them
                        pred = r['predictions'][0]['label']
                        print("Put_space : ",str(put_space))
                        if len(sentence_all)>2 and pred == sentence_all[-2] and pred == sentence_all[-1] :
                            if pred =="STOP":
                                print("Stopping...")
                                break
                            elif pred.lower() =="OK GOOGLE".lower():
                                sentence="Ok Google , "
                                sentence_all=[]
                            else:
                                if put_space:
                                    sentence = sentence + " " + pred
                                    put_space = False
                                else: 
                                    sentence = sentence + pred
                                sentence_all=[]
                        else:
                            sentence_all.append(pred)
                        print("Pred :",pred)
                        print("Sentence :", sentence)
                    # otherwise, the request failed
                    else:
                        print("Request failed")
                read_end = time.time()
            except:
                print("Image not readable")
        else:
            frames=0
    return sentence
sentence = detect_sign_language()
try:
    reply = get_assistant(sentence)
except:
    print("No reply from Assistant")

Put_space :  True
Pred : T
Sentence : 
Put_space :  True
Pred : V
Sentence : 
Put_space :  True
Pred : V
Sentence : 
Put_space :  True
Pred : V
Sentence :  V
Put_space :  False
Pred : D
Sentence :  V
Put_space :  False
Pred : D
Sentence :  V
Put_space :  False
Pred : L
Sentence :  V
Put_space :  False
Pred : A
Sentence :  V
Put_space :  False
Pred : A
Sentence :  V
Put_space :  False
Pred : B
Sentence :  V
Put_space :  False
Pred : T
Sentence :  V
Put_space :  False
Pred : OK GOOGLE
Sentence :  V
Put_space :  False
Pred : OK GOOGLE
Sentence :  V
Put_space :  False
Pred : OK GOOGLE
Sentence : Ok Google , 
Put_space :  False
Pred : OK GOOGLE
Sentence : Ok Google , 
Put_space :  False
Pred : T
Sentence : Ok Google , 
Put_space :  False
Pred : T
Sentence : Ok Google , 
Put_space :  False
Pred : W
Sentence : Ok Google , 
Put_space :  False
Pred : W
Sentence : Ok Google , 
Put_space :  False
Pred : W
Sentence : Ok Google , W
Put_space :  False
Pred : W
Sentence : Ok Google , W
Put_space :  F